In [1]:
import sys
sys.path.append('..')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
# lang = 'zh-cn' if gezi.in_notebook() else sys.argv[1]
lang = 'fr' if gezi.in_notebook() else sys.argv[1]
lang_ = lang.split('-')[-1]
ic(lang, lang_)

[09/08/22 00:46:00] 56439108.py:4 in <module>- lang: 'fr', lang_: 'fr'


('fr', 'fr')

In [3]:
from nltk.tokenize import sent_tokenize
from googletrans import Translator
import time
trans = Translator()
ic(trans.translate('照片').text)

[09/08/22 00:46:06] 2180798362.py:5 in <module>
                    trans.translate('照片').text: 'photo'


'photo'

In [4]:
def translate(text, lang='zh-cn', max_len=512):
  num_words = len(text.split())
  if num_words <= max_len:
    mid_text = trans.translate(text, dest=lang).text
    target_text = trans.translate(mid_text, dest='en').text
    time.sleep(0.5)
  else:
    mid_texts = []
    target_texts = []
    sents = sent_tokenize(text)    
    num_sents = len(sents)
    parts = int(num_words / max_len) + 1
    if num_sents > parts:
      sents = np.array_split(sents, parts)
      for i in range(len(sents)):
        sents[i] = ''.join(sents[i])
    for sent in sents:
      # print(sent)
      mid_text = trans.translate(sent, dest=lang).text
      # print(mid_text)
      target_text = trans.translate(mid_text, dest='en').text
      # print(target_text)
      time.sleep(0.5)
      mid_texts.append(mid_text)
      target_texts.append(target_text)
    mid_text = ''.join(mid_texts)
    target_text = ''.join(target_texts)
  return mid_text, target_text

In [5]:
trans_dict = {}
trans_dict2 = {}

In [6]:
def translate_(text, lang):
  if text not in trans_dict or text not in trans_dict2:
    try:
      mid_text, target_text = translate(text, lang)
      trans_dict[text] = mid_text
      trans_dict2[text] = target_text
    except Exception as e:
      # ic(e, len(text.split()))
      # bads.append(text)
      time.sleep(1)
      try:
        mid_text, target_text = translate(text, lang, 256)
        trans_dict[text] = mid_text
        trans_dict2[text] = target_text
      except Exception as e:
        time.sleep(1)
        try:
          mid_text, target_text = translate(text, lang, 128)
          trans_dict[text] = mid_text
          trans_dict2[text] = target_text
        except Exception as e:
          time.sleep(1)
          try:
            mid_text, target_text = translate(text, lang, 64)
            trans_dict[text] = mid_text
            trans_dict2[text] = target_text
          except Exception as e:
            ic(e, len(text.split()))

In [7]:
def gen_essay(mark='train'):
  essay_ids, texts = [], []
  root = FLAGS.root_v1
  def _read(file):
    essay_id = os.path.basename(file)[:-4]
    return essay_id, open(file).read().strip()
  res = gezi.prun(_read, glob.glob(f'{root}/{mark}/*.txt'), 20, desc='read essaies')
  essay_ids, texts = list(zip(*res))
  df = pd.DataFrame({
    'essay_id': essay_ids,
    'essay_text': texts
  })
  df['total_essay_words'] = df.essay_text.apply(lambda x: len(x.split()))
  # ic(mark, df.total_essay_words.describe())
  return df

In [8]:
d = gen_essay()
d

read essaies:   0%|          | 0/780 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

,essay_id,essay_text,total_essay_words
0,4B22D00F9BD7,"Students of the United States of America have you ever wondered what it would be like to do school work from home rather than actually go to the school and learn there. Students in college have had the option to take online courses instead of actually attending a college class directly for a while now. Now some high schools are beginning to offer up a similar program to students as of recently. Students it is a lot better to do school work from home because students would feel better in a familiar environment therefore allowing them to work more efficiently, a school could also save much m...",668
1,1F8DDCFCA766,"Have you ever awoken to the arrogant, angry shriek of your alarm clock on a Monday morning? Maybe you have even experienced the arctic splash of freezing cold water falling over your back as your mom struggled to pry you from a deep sleep. It is apparent that waking up for school each morning is one of the most dreadful moments a student will face. A potential solution to this issue is the introduction of online classes and video conferences. Through this much more compatible, modern way of learning, students would be less restricted and undoubtedly happier. By establishing the opportunity...",783
2,ED27B5020694,"The Battle for Control\n\nIn the near future, driveless cars will become an inevitable reality. Their a supposed solution for most traffic problems the world posseses today. No need to worry about drunk and reckless drivers anymore, the answer is finally here! Well, dive a little deeper and you'll figure out that driveless cars will create just as many problems as they would, in theory, fix. The idea of a world with driveless cars as the main priority just runs shivers down my spine...\n\nThe main problem with driveless cars is a very nightmarish one. What if the car malfunctions, and goes...",430
3,B1AF3AD74555,"In the article the author talks about machines and computers that can tell what a person is feeling like. The author does make very valid points in the article, explaining and giving well thought out details about the process as it helps us readers understand. The author claims that this machine can tell what you are feeling by ur facial structure in the moment and how your cheek bones and eyebrows and other parts of your face affect that. This is very valuable because the author has plenty of evidence behind it all. An example is the Mona LIsa. They inspected all of her bones in her face ...",478
4,F0B1C389A815,Have you ever needed some dire advice for something urgent and asked only one person and wondered why things went wrong. Asking one person is not safe. Some people could mislead you on accident or on purpose. In other cases some people are just not experienced in the situation. Well in these situations you should always ask multiple people.\n\nMore opinions give you more options to choose from. Having more options can help you come to the solutions to your problem much easier. You can pick the option that you are most comfortable with and sounds the best. Which also makes you more confiden...,319
...,...,...,...
15589,AB34D8C23C7F,"The author thinks that Venus is a worthy pursuit to discover and explore. It would be a worthy goal for humans to evolve and grow as a species and to understand that we're able to understand how to universe works. The author does claim that the study of Venus is important and that we should take the time to study and understand it better.\n\nThe first thing the author mentions about this is in paragraph four. In this paragraph the author talks about how astronomers are fascinated by Venus. He says, ""Astronomers are fascinated by Venus because it may well have once have been the most Earth-...",429
15590,3AFA78FFFC15,So I would be talking about how people should sign up for something at there own school because people should do something they love thats my first reason. Everyone should be doing something its no

In [9]:
d['full_text'] = d.essay_text.apply(resolve_encodings_and_normalize)

In [10]:
gezi.prun_loop(lambda x: translate_(x, lang), d.full_text.values, 1, desc='trans')

trans:   0%|          | 0/15594 [00:00<?, ?it/s]

In [ ]:
# ic(len(trans_dict), len(d))
# assert len(trans_dict) == len(d)

[09/08/22 00:10:48] 571746545.py:1 in <module>
                    len(trans_dict): 15593
                    len(d): 15594


AssertionError: 

In [ ]:
d[f'{lang_}_text'] = d.full_text.apply(lambda x: trans_dict[x])

In [ ]:
d[f'trans_{lang_}'] = d.full_text.apply(lambda x: trans_dict2[x])

In [ ]:
gezi.set_pd_widder()

In [ ]:
d

essay_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
d.to_csv(f'{FLAGS.root}/train_{lang_}_v1.csv', index=False)